INSTALLING DEPENDENCIES

In [ ]:
!pip install ultralytics huggingface_hub supervision pytesseract opencv-python-headless
!apt-get install -y tesseract-ocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.4/904.4 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 9.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,244 kB/s)
Selecting

IMPORTING LIBRARIES

In [ ]:
import pandas as pd
from datetime import datetime
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
from supervision import Detections
import pytesseract
import cv2
import numpy as np
from PIL import Image
from google.colab import files
from IPython.display import display
from ipywidgets import Button, FileUpload, Image as ipyImage
import io
import re


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Initializing YOLO Model

In [ ]:
# Initialize YOLO model
repo_config = dict(
    repo_id="arnabdhar/YOLOv8-nano-aadhar-card",
    filename="model.pt",
    local_dir="./models"
)
model = YOLO(hf_hub_download(**repo_config))
id2label = model.names


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


model.pt:   0%|          | 0.00/6.26M [00:00<?, ?B/s]

Calculate Age from Date of Birth

In [ ]:
def calculate_age(dob_str):
    """Calculate age from date of birth string."""
    # clean9jng the DOB string
    dob_str = re.sub(r'^DOB:\s*', '', dob_str.strip())

    try:
        # parsing
        if len(dob_str.split('/')[-1]) == 2:
            dob = datetime.strptime(dob_str, '%d/%m/%y')
        else:  # if year in yyyy format
            dob = datetime.strptime(dob_str, '%d/%m/%Y')


        today = datetime.now()


        age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
        return age
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None


Check Eligibility Based on Age and Gender

In [ ]:
def check_eligibility(age, gender, criterion):
    """Check if person meets the eligibility criterion."""
    # if criterion is gender-based
    if criterion.lower() in ['male', 'female']:
        return gender.lower() == criterion.lower()

    # if criterion is age-based
    try:

        age_range = criterion.split('-')
        min_age = int(age_range[0])
        max_age = int(age_range[1])
        return min_age <= age <= max_age
    except:
        return False


Get Eligible Schemes

In [ ]:
def get_eligible_schemes(age, gender, schemes_df):
    """Get list of schemes for which the person is eligible."""
    eligible_schemes = []

    for _, scheme in schemes_df.iterrows():
        if check_eligibility(age, gender, scheme['Criterion']):
            eligible_schemes.append({
                'name': scheme['Scheme Name'],
                'description': scheme['Description']
            })

    return eligible_schemes


Extract Text from Image

In [ ]:
def extract_text_from_image(img):
    """Extract text from image using YOLO and OCR."""
    detections = Detections.from_ultralytics(model.predict(img)[0])
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    extracted_text = {}

    for i, detection in enumerate(detections.xyxy):
        x1, y1, x2, y2 = map(int, detection)
        cropped_img = img_cv[y1:y2, x1:x2]
        text = pytesseract.image_to_string(cropped_img, config='--psm 6').strip()
        class_name = detections.data['class_name'][i]
        extracted_text[class_name] = text

    return extracted_text


Process Image and Display Results

In [ ]:
def process_and_display_results(img, schemes_df):
    """Process image and display extracted information and eligible schemes."""
    # extract text from image
    extracted_text = extract_text_from_image(img)

    print("\nExtracted Information from Aadhaar Card:")
    for label, text in extracted_text.items():
        print(f"{label}: {text}")


    age = None
    if 'DATE_OF_BIRTH' in extracted_text:
        age = calculate_age(extracted_text['DATE_OF_BIRTH'])
        if age is not None:
            print(f"\nCalculated Age: {age} years")
        else:
            print("\nCouldn't calculate age: Invalid DOB format")

    gender = extracted_text.get('GENDER', '').strip()


    if age is not None and gender:
        eligible_schemes = get_eligible_schemes(age, gender, schemes_df)

        if eligible_schemes:
            print("\nEligible Schemes:")
            for scheme in eligible_schemes:
                print(f"\nScheme Name: {scheme['name']}")
                print(f"Description: {scheme['description']}")
        else:
            print("\nNo eligible schemes found for this person.")
    else:
        if age is None:
            print("\nCouldn't calculate age from the given DOB")
        if not gender:
            print("\nCouldn't extract gender information")


Handle Image Upload

In [ ]:
def upload_image():
    """Handle image upload."""
    uploaded = files.upload()
    image_path = next(iter(uploaded))
    return Image.open(image_path)


Capture Image from Webcam

In [ ]:
def capture_image():
    """Capture image from webcam."""
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Press q to capture', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    return Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


Display Buttons for Upload or Capture

In [ ]:
def display_buttons(schemes_df):
    """Display upload and capture buttons."""
    upload_button = Button(description="Upload Image")
    upload_button.on_click(lambda x: process_and_display_results(upload_image(), schemes_df))

    capture_button = Button(description="Capture Image")
    capture_button.on_click(lambda x: process_and_display_results(capture_image(), schemes_df))

    display(upload_button, capture_button)


Main Execution

In [ ]:
# Main execution
if __name__ == "__main__":

    try:
        schemes_df = pd.read_csv('schemes.csv', encoding='latin-1')  # or 'Windows-1252', 'ISO-8859-1'
        print("Loaded schemes data successfully")
    except Exception as e:
        print(f"Error loading schemes data: {e}")

        schemes_df = pd.DataFrame({
            'Scheme Name': ['Young Adult Scheme', 'Male Health Program'],
            'Criterion': ['25-35', 'Male'],
            'Description': ['Benefits for young adults', 'Health benefits for males']
        })

    #display button
    display_buttons(schemes_df)


Loaded schemes data successfully


Button(description='Upload Image', style=ButtonStyle())

Button(description='Capture Image', style=ButtonStyle())

Saving random aadha.jpg to random aadha.jpg

0: 448x640 1 AADHAR_NUMBER, 1 DATE_OF_BIRTH, 1 GENDER, 1 NAME, 300.5ms
Speed: 28.2ms preprocess, 300.5ms inference, 31.8ms postprocess per image at shape (1, 3, 448, 640)

Extracted Information from Aadhaar Card:
GENDER: MALE
AADHAR_NUMBER: 9183 0074 6619
NAME: Prakash Ranjan
DATE_OF_BIRTH: DOB: 05/07/1994

Calculated Age: 30 years

Eligible Schemes:

Scheme Name: Abcd
Description: A scheme to provide financial assistance to non-pensioner Ex-Servicemen up to rank of Havildar /equivalent and their widows who are living in extreme poverty.

Scheme Name: Kaddu
Description: You can Die
